This notebook brings uses the scripts read_data_features.py and read_data_genre_track.py to arrive at a cleaned dataset with ready to use features and labels.

The following is 'read_data_genre_track.py'. 

In [2]:
import numpy as np
import pandas as pd

In [3]:
metadata_path = 'fma_metadata'

In [4]:
## Read genre information
genres = pd.read_csv(metadata_path+'\\genres.csv')
genre_grps = genres.groupby('parent')
genre_id_root = genre_grps.get_group(0)['genre_id'] # Extract the parent groups


In [6]:
## Read track information
fields = ['track']
tracks = pd.read_csv(metadata_path+'\\tracks_mod.csv',nrows=85000)
tracks = tracks.dropna()
tgen = list(tracks['genres'])
tgen = [t.replace('[','') for t in tgen]
tgen = [t.replace(']','') for t in tgen]
tgen = [t.replace(' ','') for t in tgen]
for k in  range(len(tgen)):
    if(len(tgen[k])!=0):
        tgen[k] = [i for i in map(int, tgen[k].split(','))]
    else:
        tgen[k]=[]
tracks['genres']=tgen

In [7]:
## Match the tracks to the genre data: Create dictionary with genre label as key
genre_dict = {}
for i in range(len(tgen)):
    if i in tracks.index:
        ti = tracks['track_id'][i]
        for k in tgen[i]:
            if k is not None:
                if (k in genre_dict.keys()):
                    genre_dict[k]+=[ti]
                else:
                    genre_dict.update({k:[ti]})

In [8]:
## Total number of samples for the different root genres
for i in genre_id_root:
    print(i,len(genre_dict[i]))
    

2 1571
3 1438
4 2040
5 1735
8 852
9 703
10 5215
12 6629
13 137
14 386
15 18143
17 5889
20 292
21 4815
38 18376
1235 2912


The following is 'read_data_features.py'.

In [9]:
## Column information
fields = ['mfcc.40','mfcc.42','mfcc.43','mfcc.44','mfcc.45','mfcc.48','mfcc.49','mfcc.50',\
    'mfcc.53','mfcc.56','mfcc.120','mfcc.121','mfcc.122','mfcc.124','spectral_contrast.14',\
        'spectral_contrast.15','spectral_contrast.16','spectral_contrast.43',\
            'spectral_contrast.44','feature']


In [10]:
## Read feature information 
features = pd.read_csv(metadata_path+'\\features.csv',usecols=fields)
feat_temp = features['feature']
for i in range(3,len(feat_temp)):
    feat_temp[i]=int(feat_temp[i])
features['feature'] = feat_temp

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,293,295,296,297,298,301,302,303,306,309,373,374,375,377,428,429,430,457,458) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
## Include code to filter the rows based on the training id
temp_id_set =[2,3,5,10,20,26]
feat1 = features.loc[features['feature'].isin(temp_id_set)]